# 영상처리
### 1. 영상 30초 분할, 소리제거
### 2. 영상 프레임 저장(모델에서 해주는 부분이므로 하지 않아도 됨 테스트용)
### 3. 영상의 음악 저장, 음악 분할

# 음악처리
### 4. 음악 30초 분할

# 라이브러리, 경로 지정

In [ ]:
# dependancies
# ****************************************************************************
#  pip install ffmpeg
#  pip install pydub

import subprocess
import os
import re

In [ ]:
# video가 포함된 directory의 경로 설정
# ****************************************************************************

path_dir = " "

file_list=os.listdir(path_dir) # 디렉토리에 들어있는 것들 리스트를 받아옴(처리하려는 영상 이외의 것이 있으면 리스트에서 빼주는 전처리 필요)
file_list.sort() #이름순으로 정렬

In [ ]:
# 확장자(mp4)를 제외한 비디오 제목만 남기기
# ****************************************************************************
video_list=[]
for i in file_list:
    i_split=i.split(".")
    video_list.append(i_split[0])

# print(video_list)
print(len(video_list))

In [ ]:
# 영상 길이 구하기(초)  - 비디오의 소리 분할시에 이 길이 활용
# ************************************************************************
video_time_list = []

for video in video_list:
    result = subprocess.Popen('ffprobe -i "{}\\{}.mp4" -show_entries format=duration -v quiet -of csv="p=0"'.format(path_dir, video), stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    output = result.communicate()
    # print(output[0])
    video_time = re.findall("\d+",str(output[0]))[0] # 정규식, 정수단위의 초만 남김
    # print(video_time)
    video_time_list.append(video_time)
# print(video_time_list)
print(len(video_time_list))

# 영상 처리 : 소리제거+30초분할/프레임/소리 따로 저장

## 0. 영상 저장 위치 지정

In [ ]:
# 영상처리하고 저장할 위치 지정
# ****************************************************************************
paht_save_v = ""

## 1. 영상 저장 - 영상 소리제거+30초로 분할

In [ ]:
# 영상 소리제거, 30초 분할하여 저장

for idx, j in enumerate(video_time_list): # 영상 길이를 받아옴
    order = int(j)//30 # 30초 단위로 잘라 떨어지지 않는 부분은 버림 - 몫
    for k in range(order): # 파일명을 위한 k
        # print(idx, j, order, k, k*30)
        # print(video_list[idx])
        command_v = 'ffmpeg -ss "{}" -t 30 -i "{}\\{}.mp4" -an "{}\\{}_{}.mp4"'.format(k*30, path_dir, video_list[idx], path_save_v, video_list[idx], k+1) # -ss 시작시간 / -t 시간 30초 / -i input 파일 / -an 오디오는 제거 / 마지막에 파일이 저장될 경로 지정해줌 // 영상의 vcodec을 넣으면 30초로 딱 잘리지 않아 태그 삭제함(but 시간이 3배 이상 오래 걸림)
        subprocess.call(command_v, shell=True) # shell에서 커맨드 실행
        print("command is   =======   ", command_v)

## 2. 영상 프레임 뽑기 - 필요에 따라 경로 바꿀 것

In [ ]:
# 자르려는 영상이 있는 경로 - 30초 분할된 영상이 있는 경로 지정
path_video = ''

# 영상의 프레임을 뽑은 후 저장할 경로 지정
path_frame = ''

In [ ]:
# 영상 프레임 뽑기
file_list=os.listdir(path_video)
file_list.sort()

# mp4를 제외한 비디오 제목만 남기기 - 영상 원본이 아니고 분할된 영상이므로
video_list=[]
for i in file_list:
    i_split=i.split(".")
    video_list.append(i_split[0])

# print(video_list)
print(len(video_list))

for video in video_list:
    command = 'ffmpeg -ss 00:00 -t 30 -i "{}\\{}.mp4" -r 6 -s 640*360 -qscale:v 2 -f image2 "{}\\{}_%3d.jpg"'.format(path_video, video, path_frame, video) # -r은 1초에 6장 -s 사이즈 -qscale:v 화질관련 -f 포맷:이미지
    subprocess.call(command, shell=True)
    print("command is   =======   ", command)

## 3. 영상의 소리 추출, 분할 30초씩 - 원본 영상 이용(처리한 영상과 프레임에는 소리X)

In [ ]:
# 원본 영상의 경로를 다시 확인(앞 코드와 중복됨)
# **************************************************************************
# # from pydub import AudioSegment
# import os
# import subprocess

path_dir = "" #영상 경로 지정

# file_list=os.listdir(path_dir)
# file_list.sort()
# len(file_list)

# video_list=[]
# for i in file_list:
#     i_split=i.split(".")
#     video_list.append(i_split[0])
    
# video_time_list = []

#영상 길이 구하기(초)
# for video in video_list:
#     result = subprocess.Popen('ffprobe -i "{}\\{}.mp4" -show_entries format=duration -v quiet -of csv="p=0"'.format(path_dir, video), stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
#     output = result.communicate()
#     # print(output[0])
#     video_time = re.findall("\d+",str(output[0]))[0] #정규식, 정수단위의 초만 남김
#     # print(video_time)
#     video_time_list.append(video_time)

print("video_list includes the following videos : \n", video_list)
print("the length of video_list : \n", len(video_list))
print("video_time_list includes the following time of videos : \n", video_time_list)
print("the length of video_time_list : \n", len(video_time_list))

# 음악 처리

## 4. 음악 30초 분할

In [ ]:
import subprocess
import os
import re
from mutagen.mp3 import MP3
from mutagen.asf import ASF

In [ ]:
# 음악 파일 위치 # 앞의 과정과 별개라 path_dir라는 변수를 새로 써줌 영상을 다시 처리한다면 주의할 것
musicpath = ''
path_dir = musicpath
# path_dir = musicpath_classicall
# path_dir = musicpath_dance
# path_dir = musicpath_rock

music_file_list=os.listdir(path_dir) # 디렉토리에 들어있는 것들 리스트를 받아옴(처리하려는 음악 이외의 것이 있으면 리스트에서 빼주는 전처리 필요)
music_file_list.sort() #이름순으로 정렬
len(music_file_list)
music_file_list[0]

In [ ]:
# 제목만 남기기
music_list=[]
for i in music_file_list:
    i_split=i.split(".")
    music_list.append(i_split[0])

print(music_list[0])
print(len(music_list))

# 음악 파일 길이
# 길이를 가지고 분할 처리, 파일명 지정

In [ ]:
# 음악 길이 받기
# dependancies
# ****************************************************************************
# pip install mutagen

import mutagen
from mutagen.mp3 import MP3
from mutagen.asf import ASF

path_dir_music = '' #필요에 따라 경로 지정

music_length = []

for i in music_file_list:
    i_split=i.split(".")
    music_list.append(i_split[0])
    if i_split[1]=='mp3':
        audio = MP3(path_dir_music+i)
    else:
        audio = mutagen.File(path_dir_music+i)
    length = audio.info.length
    music_length.append(length)

    print(len(music_length))
print(len(music_file_list))

## 분할 후 저장

In [ ]:
# 저장 위치
path_save_m = '' # 필요에 따라 경로 수정

# 음악 자르기
from pydub import AudioSegment
for idx, j in enumerate(music_length): # 음악의 길이는 영상 길이와 같음
    file = path_dir+"\\"+music_file_list[idx]
    savefile = path_save_m+"\\"+music_list[idx]
    music = AudioSegment.from_file(file)
    order = int(j)//30
    sec = 1000
    thirtysec = 30 * sec
    for k in range(order):
        start = k*30
        # print(idx, j, order, k, k*30)
        # print(video_list[idx])
        music30 = music[start:start+thirtysec] # pydub은 기준이 millisecond라서(1초는 10000 밀리세컨드)
        music30.export(path_save_m+"\\{}_{}".format(music_list[idx],k+1), format='wav', parameters=["-q:a", "10", "-ac", "1"]) # 내보내기
    print("success")